<a href="https://colab.research.google.com/github/123nishant/Google-Colab/blob/master/SeqModelling_FastAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  !curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from fastai.text import *

In [0]:
path = Config().data_path()

In [7]:
#! wget https://s3.amazonaws.com/fast-ai-nlp/giga-fren.tgz -P {path}

--2020-05-12 01:45:32--  https://s3.amazonaws.com/fast-ai-nlp/giga-fren.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.43.134
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.43.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2598183296 (2.4G) [application/x-tar]
Saving to: ‘/root/.fastai/data/giga-fren.tgz’

giga-fren.tgz       100%[===================>]   2.42G  45.0MB/s    in 54s     

2020-05-12 01:46:27 (45.6 MB/s) - ‘/root/.fastai/data/giga-fren.tgz’ saved [2598183296/2598183296]



In [0]:
#! tar xf {path}/giga-fren.tgz -C {path}

In [9]:
path = Config().data_path()/'giga-fren'
path.ls()

[PosixPath('/root/.fastai/data/giga-fren/giga-fren.release2.fixed.fr'),
 PosixPath('/root/.fastai/data/giga-fren/giga-fren.release2.fixed.en')]

In [0]:
#with open(path/'giga-fren.release2.fixed.fr') as f: fr = f.read().split('\n')

In [0]:
#with open(path/'giga-fren.release2.fixed.en') as f: en = f.read().split('\n')

In [0]:
 re_eq = re.compile('^(Wh[^?.!]+\?)')
 re_fq = re.compile('^([^?.!]+\?)')
 en_fname = path/'giga-fren.release2.fixed.en'
 fr_fname = path/'giga-fren.release2.fixed.fr'

In [0]:
 lines = ((re_eq.search(eq), re_fq.search(fq)) 
         for eq, fq in zip(open(en_fname, encoding='utf-8'), open(fr_fname, encoding='utf-8')))
 qs = [(e.group(), f.group()) for e,f in lines if e and f]

In [0]:
 qs = [(q1,q2) for q1,q2 in qs]
 df = pd.DataFrame({'fr': [q[1] for q in qs], 'en': [q[0] for q in qs]}, columns = ['en', 'fr'])
 df.to_csv(path/'questions_easy.csv', index=False)



In [15]:
path.ls()

[PosixPath('/root/.fastai/data/giga-fren/giga-fren.release2.fixed.fr'),
 PosixPath('/root/.fastai/data/giga-fren/giga-fren.release2.fixed.en'),
 PosixPath('/root/.fastai/data/giga-fren/questions_easy.csv')]


# **Load our data into a DataBunch**

Our questions look like this now:


In [16]:
df = pd.read_csv(path/'questions_easy.csv')
df.head()

,en,fr
0,What is light ?,Qu’est-ce que la lumière?
1,Who are we?,Où sommes-nous?
2,Where did we come from?,D'où venons-nous?
3,What would we do without it?,Que ferions-nous sans elle ?
4,What is the absolute location (latitude and lo...,Quelle sont les coordonnées (latitude et longi...


In [0]:
df['en'] = df['en'].apply(lambda x:x.lower())
df['fr'] = df['fr'].apply(lambda x:x.lower())

collate inputs and targets in a batch: they have different lengths so we need to add padding to make the sequence length the same;

In [0]:
def seq2seq_collate(samples, pad_idx=1, pad_first=True, backwards=False):
    "Function that collect samples and adds padding. Flips token order if needed"
    samples = to_data(samples)
    max_len_x,max_len_y = max([len(s[0]) for s in samples]),max([len(s[1]) for s in samples])
    res_x = torch.zeros(len(samples), max_len_x).long() + pad_idx
    res_y = torch.zeros(len(samples), max_len_y).long() + pad_idx
    if backwards: pad_first = not pad_first
    for i,s in enumerate(samples):
        if pad_first: 
            res_x[i,-len(s[0]):],res_y[i,-len(s[1]):] = LongTensor(s[0]),LongTensor(s[1])
        else:         
            res_x[i,:len(s[0]):],res_y[i,:len(s[1]):] = LongTensor(s[0]),LongTensor(s[1])
    if backwards: res_x,res_y = res_x.flip(1),res_y.flip(1)
    return res_x,res_y



In [19]:
doc(Dataset)

In [20]:
doc(DataLoader)

In [21]:


doc(DataBunch)

